In [1]:
import os
import sys

sys.path.append("../")

import torch

from datasets.fastspeech_dataset import (
    build_path_to_transcript_dict_libri_tts,
    FastSpeechDataset, Miniset
)
from pipelines.gst_fastspeech2 import meta_train_loop

device = "cuda" if torch.cuda.is_available() else "cpu"
device

/home/nmmy/Documents/src/DATN-VoiceCloning/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
torchvision is not available - cannot save figures


'cpu'

In [2]:
DATASET_BASE_PATH = '../data/librispeech'
# TEST_CLEAN_PATH = os.path.join(DATASET_BASE_PATH, 'test-clean')
TEST_CLEAN_PATH = '/home/nmmy/Documents/dataset/test-clean/'
TRAIN_CLEAN_PATH = os.path.join(DATASET_BASE_PATH, 'train-clean-100')
ALIGNER_CHECKPOINT = "/home/nmmy/Documents/dataset/aligner.pt"

In [3]:
transcript_dict = build_path_to_transcript_dict_libri_tts(TEST_CLEAN_PATH)

In [4]:
# only test on 100 samples
transcript_dict_100 = {}
for i,(k,v) in enumerate(transcript_dict.items()):
    if i == 100: break
    transcript_dict_100[k] = v

In [5]:
dataset = FastSpeechDataset(
    path_to_transcript_dict=transcript_dict,
    acoustic_checkpoint_path=ALIGNER_CHECKPOINT,  # path to aligner.pt
    cache_dir="./librispeech",
    lang="en",
    loading_processes=4,  # depended on how many CPU you have
    device=device,
)

Prepared a FastSpeech dataset with 18 datapoints in ./librispeech.


In [6]:
spkId = os.path.join(TEST_CLEAN_PATH, '61')

In [7]:
minidataset = Miniset(dataset, speakerID='61')

In [8]:
datasetlist = [minidataset]

In [9]:
from tts.models.fastspeech2.FastSpeech2 import FastSpeech2

In [10]:
net = FastSpeech2()

In [11]:
meta_train_loop.train_loop(net,datasetlist, 
                           device='cpu', 
                           batch_size=2,
                           save_directory='.',
                           path_to_checkpoint=None,
                           phase_1_steps=2,
                           phase_2_steps=2,
                           steps_per_checkpoint=1,
                           path_to_embed_model='/home/nmmy/Documents/dataset/embedding_function.pt',
                           lr=0.01)

/home/nmmy/Documents/src/DATN-VoiceCloning/venv/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py:120: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
  0%|          | 0/4 [00:00<?, ?it/s]/home/nmmy/Documents/src/DATN-VoiceCloning/venv/lib/python3.10/site-packages/torch/amp/autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
 25%|██▌       | 1/4 [00:03<00:10,  3.43s/it]


Total Steps: 1
Spectrogram Loss: 15.212


 50%|█████     | 2/4 [00:11<00:12,  6.44s/it]


Total Steps: 2
Spectrogram Loss: 15.266


 75%|███████▌  | 3/4 [00:15<00:05,  5.25s/it]


Total Steps: 3
Spectrogram Loss: 15.946
Cycle Loss: 1.211


100%|██████████| 4/4 [00:20<00:00,  5.15s/it]
